In [1]:
from pymongo import MongoClient
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
client = MongoClient()
db = client.get_database("companies")

In [3]:
len(list(db.companies.find({})))

18801

In [4]:
comp = db.companies

In [5]:
# I create a new collection with the offices in USA, and discard all companies with no coordinates:
res = comp.aggregate([
                      {"$unwind":"$offices" },
                      {"$match":{"offices.latitude":{"$ne":None}}},
                      {"$match":{"offices.longitude":{"$ne":None}}},
                      {"$project":{"_id":0}} 
                                             
                     ])

In [6]:
res

In [7]:
db.companies_unwinded.insert_many(res)

In [8]:
offices = db.companies_unwinded

In [9]:
for c in res:
    filt = {"_id":c["_id"]}
    coord = {
              "type": "Point",
              "coordinates": [c["offices"]["longitude"], c["offices"]["latitude"]]
            }
    update = {"$set":{"coord":coord}}
    offices.update_one(filt,update)

In [13]:
offices

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'companies'), 'companies_unwinded')

In [14]:
point = {"type":"Point",
         "coordinates":[-81.104318, 32.077052]}

query ={
    "coord":{
        "$near":{
            "$geometry":point,
            "$maxDistance":100_000,
            #"$minDistance":
        }
    }
}

res = offices.find(query,{"offices":1})

In [9]:
filter_query={"deadpooled_year":{"$ne":"null"},'offices.state_code':'CA',"category_code":{"$in":['web', 'mobile', 'games_video', 'software', 
                                                                       'photo_video', 'biotech', 'cleantech']}}
project={"offices":1}
res=offices.find(filter_query,project)